!pip install selenium
!pip install pandas

# Testing

In [42]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [44]:
# URL
url = "https://epaath.olenepal.org/subjects.html?sub=math&lang=en&grade=4"

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Use the appropriate driver for your browser
driver.get(url)

# Wait for JavaScript to load
time.sleep(5)  # Adjust the sleep time as needed

# Get the HTML content after JavaScript execution
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)
driver.quit()

<html><head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-W1QLTN88RR&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-172822248-2"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());

    gtag('config', 'UA-172822248-2');
    </script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<title>Subject: math | Lang:  en | Grade: 4</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<meta content="https://epaath.olenepal.org/images/ole_meta_logo.jpg" property="og:image"/>
<

In [45]:
all_divs = soup.find_all('div')
print("Total divs found:", len(all_divs))
for div in all_divs[:5]:  # Print first 5 divs to check
    print(div, "\n")


Total divs found: 33
<div class="terms_of_use_info" style="display: none;">
<div>
<p><span>Terms of Use</span><hr style="margin-top:8px; margin-bottom:8px;"/>E-Paath is a collection of digital learning materials that are designed and developed by OLE Nepal and distributed freely. These contents are the intellectual property of OLE Nepal, and redistributing or reposting them for monetization or without credits to OLE Nepal is strictly prohibited. <img src="images/activityguideimages/iconcross1.png"/></p>
</div>
</div> 

<div>
<p><span>Terms of Use</span><hr style="margin-top:8px; margin-bottom:8px;"/>E-Paath is a collection of digital learning materials that are designed and developed by OLE Nepal and distributed freely. These contents are the intellectual property of OLE Nepal, and redistributing or reposting them for monetization or without credits to OLE Nepal is strictly prohibited. <img src="images/activityguideimages/iconcross1.png"/></p>
</div> 

<div class="navbar colorPurple" r

In [51]:
def scrape_grade_4_math(url):
    # Sending a request to the URL
    #response = requests.get(url)
    #print("URL Fetched:", response.url)  # Check if the URL is correct
    #print("Status Code:", response.status_code)  # Check for successful response

    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  # Use the appropriate driver for your browser
    driver.get(url)

    # Wait for JavaScript to load
    time.sleep(2)  # Adjust the sleep time as needed

    # Get the HTML content after JavaScript execution
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    

    # Finding all the subject boxes
    #subject_boxes = soup.find_all('div', class_='subjectBox')
    # Finding all the subject boxes using a CSS selector
    subject_boxes = soup.select('.subjectBox.intro-subject-box')
    print("Number of Subject Boxes Found:", len(subject_boxes))  # Check how many boxes are found

     # Extracting activity names and URLs
    data = []
    for box in subject_boxes:
        activity_name = box.find('span', class_='activityname').text.strip()
        activity_url = box.find('a')['href']
        activity_thumb_src = box.find('img', class_='linkImgGrid')['src']  # Accessing the src attribute of the img tag
        #print(activity_name,activity_url,activity_thumb_src)
        full_url = url.rsplit('/', 1)[0] + '/' + activity_url  # Construct the full URL (online, change domain for offline to server IP)
        full_thumb_src =  url.rsplit('/', 1)[0] + '/' + activity_thumb_src
        
        data.append([activity_name, '4', 'Math', full_thumb_src , full_url])

    return data

# URL for Grade 4 Math
url = "https://epaath.olenepal.org/subjects.html?sub=math&lang=en&grade=4"

# Scrape data from the URL
grade_4_math_data = scrape_grade_4_math(url)

# Creating a DataFrame
df = pd.DataFrame(grade_4_math_data, columns=['Activity Name','Grade', 'Subject', 'Thumbnail', 'URL'])
print(df)

Number of Subject Boxes Found: 13
                      Activity Name Grade Subject  \
0                            Angles     4    Math   
1                      Solid shapes     4    Math   
2              Rounding off numbers     4    Math   
3                          Fraction     4    Math   
4               Comparing Fractions     4    Math   
5                     Mixed Numbers     4    Math   
6                        Percentage     4    Math   
7                   Bill and budget     4    Math   
8           Introduction to Algebra     4    Math   
9   Introduction to Algebraic Terms     4    Math   
10  Solve Basic Algebraic Equations     4    Math   
11                              Set     4    Math   
12      Prime and composite numbers     4    Math   

                                            Thumbnail  \
0   https://epaath.olenepal.org/images/thumb/math/...   
1   https://epaath.olenepal.org/images/grade4/math...   
2   https://epaath.olenepal.org/images/grade4/math..

# Script to generate a csv of all E-Paath activities

In [52]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

## Run only once. Use saved csv. 

In [60]:

def generate_epaath_urls(base_url, grades, subjects):
    urls = []
    for grade in grades:
        for subject in subjects:
            # Exclude Nepali for Grades 5, 6, 7, 8
            if subject == 'nepali' and grade in [5, 6, 7, 8]:
                continue

            # Set language based on subject
            if subject == 'nepali':
                languages = ['np']
            elif subject == 'english':
                languages = ['en']
            else:  # For Math and Science subjects
                languages = ['en', 'np']

            # Generate URLs for each language
            for language in languages:
                url = f"{base_url}/subjects.html?sub={subject}&lang={language}&grade={grade}"
                urls.append((grade, subject, language, url))

    return urls

def scrape_activities(url, grade, subject):
    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  # Use the appropriate driver for your browser
    driver.get(url)

    # Wait for JavaScript to load
    time.sleep(3)  # Adjust the sleep time as needed

    # Get the HTML content after JavaScript execution
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Finding all the subject boxes using a CSS selector
    subject_boxes = soup.select('.subjectBox.intro-subject-box')

    # Extracting activity names and URLs
    data = []
    for box in subject_boxes:
        activity_name = box.find('span', class_='activityname').text.strip() #get activity name
        activity_url = box.find('a')['href'] #get activity link
        activity_thumb_src = box.find('img', class_='linkImgGrid')['src']  # Accessing the src attribute of the img tag
        full_url = url.rsplit('/', 1)[0] + '/' + activity_url  # Construct the full URL (change domain to serverIP for offline)
        full_thumb_src = url.rsplit('/', 1)[0] + '/' + activity_thumb_src  # Construct the full URL for thumbnail
        data.append([activity_name, grade, subject, full_thumb_src, full_url])

    driver.quit()
    return data#

In [61]:
# Base URL and URL generation
base_url = "https://epaath.olenepal.org"
grades = range(1, 9)  # Grades 1 to 8
print("grades:",grades);
subjects = ['math', 'science', 'english', 'nepali']
epaath_urls = generate_epaath_urls(base_url, grades, subjects)

#epaath_urls = epaath_urls[0:2] #test using few links
print("No of urls: %d \n"%len(epaath_urls))

# Dataframe to store all activities
all_activities = pd.DataFrame(columns=['Activity Name', 'Grade', 'Subject', 'Thumbnail', 'URL'])
print("yo yo:", all_activities)
# Iterate over each URL and scrape data
count = 0 
for grade, subject, language, url in epaath_urls:
    count = count+1;
    print("No %d =>"%count,"Getting activities for Grade: %d, Subject: %s and Language: %s \n"%(grade, subject, language))
    activities_data = scrape_activities(url, grade, subject)
    activities_data = pd.DataFrame(activities_data, columns=['Activity Name', 'Grade', 'Subject', 'Thumbnail', 'URL'])
    all_activities = pd.concat([all_activities,pd.DataFrame(activities_data, columns=['Activity Name', 'Grade', 'Subject', 'Thumbnail', 'URL'])], ignore_index=True)
    
# Save the DataFrame to a CSV file
csv_filename = "epaath_activities_may2024.csv"
all_activities.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")


grades: range(1, 9)
No of urls: 44 

yo yo: Empty DataFrame
Columns: [Activity Name, Grade, Subject, Thumbnail, URL]
Index: []
No 1 => Getting activities for Grade: 1, Subject: math and Language: en 

No 2 => Getting activities for Grade: 1, Subject: math and Language: np 

No 3 => Getting activities for Grade: 1, Subject: science and Language: en 

No 4 => Getting activities for Grade: 1, Subject: science and Language: np 

No 5 => Getting activities for Grade: 1, Subject: english and Language: en 

No 6 => Getting activities for Grade: 1, Subject: nepali and Language: np 

No 7 => Getting activities for Grade: 2, Subject: math and Language: en 

No 8 => Getting activities for Grade: 2, Subject: math and Language: np 

No 9 => Getting activities for Grade: 2, Subject: science and Language: en 

No 10 => Getting activities for Grade: 2, Subject: science and Language: np 

No 11 => Getting activities for Grade: 2, Subject: english and Language: en 

No 12 => Getting activities for Grade

In [62]:
print(len(all_activities))

860


In [63]:
all_activities.head()

,Activity Name,Grade,Subject,Thumbnail,URL
0,Geometric Shapes,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matg...
1,Numbers from 1 to 5,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matn...
2,Basic Addition,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=mata...
3,Subtraction up to 20,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matb...
4,One to twenty,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=mato...


In [64]:
all_activities.describe()

,Activity Name,Grade,Subject,Thumbnail,URL
count,860,860,860,860,860
unique,760,8,4,535,860
top,ऐकिक नियम,2,science,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matg...
freq,4,144,328,6,1


In [66]:
# Convert the DataFrame to JSON format
all_activities.to_json('epaath_activities_may2024.json', orient='records')

## Load Saved CSV 

In [65]:
# Load the CSV file into a DataFrame
all_activities_loaded = pd.read_csv(csv_filename)

print("No of activities: %d \n"%len(all_activities_loaded))
# Display the first few rows of the DataFrame
all_activities_loaded.head()

No of activities: 860 



,Activity Name,Grade,Subject,Thumbnail,URL
0,Geometric Shapes,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matg...
1,Numbers from 1 to 5,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matn...
2,Basic Addition,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=mata...
3,Subtraction up to 20,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=matb...
4,One to twenty,1,math,https://epaath.olenepal.org/images/thumb/math/...,https://epaath.olenepal.org/start.html?id=mato...
